# Web scraping

## Instalación de paquetes

Para este proceso se usó como referencia un tutorial para hacer [web scraping](https://nanonets.com/blog/web-scraping-with-python-tutorial/).

In [1]:
import requests # Para hacer solicitudes al sitio web
from bs4 import BeautifulSoup # Para analizar el código del HTML
import pandas as pd
import time # Para añadir un cierto delay para no sobrecargar la página con las solucitudes
import numpy as np

## Extracción de los datos

Se inicia el proceso de extracción de datos, para esto se eligió una página web de [compra/venta de carros](https://crautos.com/autosusados/), se usará el comando [find_all](https://j2logo.com/python/web-scraping-con-python-guia-inicio-beautifulsoup/#scraping-bs-pasos) de la librería BeautifulSoup, además, se buscó la manera de hacer casting de [string a float](https://ellibrodepython.com/casting-python#:~:text=class%20%27str%27>-,Convertir%20string%20a%20float,a%20float%20usando%20float()%20.) pues se ocupará para los precios.

In [2]:
# Se selecciona el url del que se estraerá la información
url = "https://crautos.com/autosusados/"

# Manda una solicitud para obtener la información de la página web (librería requests)
respuesta = requests.get(url)

# Se analiza la información del HTML (librería BeautifulSoup)
sopa = BeautifulSoup(respuesta.content, 'html.parser')

# Se inicia extrayendo el tipo de cambio
tipo_cambio = 0

for fila in sopa.find_all('div', class_='brandtitle2'):
    if(fila.get_text().count("Tipo de cambio: ") > 0):
        tipo_cambio = float(fila.get_text().split("¢")[1][0:5])

In [3]:
# Se crea una lista vacía para guardar los otros elementos relevantes de la página web
marcas = []
modelos = []
annos = []
transmisiones = []
pasajeros = []
precio = []

# Iteramos por las filas de código del HTML para conseguir los elementos que nos interesan, iniciando por la marca del carro
for fila in sopa.find_all('span', class_='brandtitle'):
    marca = fila.get_text()
    marcas.append(marca)
    time.sleep(1)

# Hay 2 nombres de la marca, entonces agragamos los restantes
for fila in sopa.find_all('td', class_='brandtitle2'):
    marca = fila.get_text()
    marcas.append(marca)
    time.sleep(1)

# Seguimos con el modelo y el año
for fila in sopa.find_all('td', class_='modeltitle'):
    anno = fila.find('b').get_text()[ : 4] # Extrae una cadena con muchos espacios, solo nos interesa el año (los primeros 4 caracteres)
    modelo = fila.get_text().strip().split(' - ')[0] # Extrae toda la información detallada del carro, quita los espacios del inicio y el final, separa por guiones y guarda el primer elemento
    annos.append(anno)
    modelos.append(modelo)
    time.sleep(1)

# Luego, la transmisión y el número de pasajeros
for fila in sopa.find_all('div', class_='transtitle'):
    # Guardamos la fila, pues se le harán
    fila_actual = fila.get_text()
    
    # Obtenemos la transmisión
    if(fila_actual.count("Automático") > 0):
        transmision = "Automático"
    elif(fila_actual.count("Manual\r\n") > 0):
        transmision = "Manual"
    else:
        transmision = np.nan
    transmisiones.append(transmision)
    
    # Obtenemos los pasajeros
    if(fila_actual.count("2 pas.") > 0):
        pasajero = 2
    elif(fila_actual.count("4 pas.") > 0):
        pasajero = 4
    elif(fila_actual.count("5 pas.") > 0):
        pasajero = 5
    elif(fila_actual.count("6 pas.") > 0):
        pasajero = 6
    elif(fila_actual.count("7 pas.") > 0):
        pasajero = 7
    elif(fila_actual.count("8 pas.") > 0):
        pasajero = 8
    elif(fila_actual.count("10 pas.") > 0):
        pasajero = 10
    else:
        pasajero = np.nan
    pasajeros.append(pasajero)
    time.sleep(1)

# Extraemos el precio
for fila in sopa.find_all('td', class_='precio'):
    # Removemos las comas de los precios
    fila_limpia = fila.get_text().replace(",", "")
    
    # Ponemos el precio, en caso de que esté en dólares se multiplica por el tipo de cambio de la página
    if(fila_limpia.count("¢") > 0):
        coste = float(fila_limpia.split("¢ ")[1][0:8])
    elif(fila_limpia.count("$") > 0):
        coste = float(fila_limpia.split("$")[1][0:5])
        coste = coste * tipo_cambio
    else:
        coste = np.nan
    precio.append(coste)
    time.sleep(1)

# Para la siguiente parte, se encuentran algunos repetidos, por lo que se hará un set para evitar este problema
lista_sec = set(sopa.find_all('span', class_='precio'))

for fila in lista_sec:
    #Removemos las comas de los precios
    fila_limpia = fila.get_text().replace(",", "")
    
    if(fila_limpia.count("¢") > 0):
        coste = float(fila_limpia.split("¢")[1][1:8])
    elif(fila_limpia.count("$") > 0):
        coste = float(fila_limpia.split("$")[1][0:5])
        coste = coste * tipo_cambio
    else:
        coste = np.nan
    precio.append(coste)
    time.sleep(1)

In [4]:
df_carros = pd.DataFrame(
    {"Marca": marcas,
     "Modelo": modelos, 
     "Años": annos, 
     "Transmisiones": transmisiones[0:62],
     "Pasajeros": pasajeros[0:62],
     "Precio": precio[0:62]})
df_carros

,Marca,Modelo,Años,Transmisiones,Pasajeros,Precio
0,Toyota,COROLLA S,2010,Automático,5,6500000.0
1,BMW,X3,2020,Automático,5,23000000.0
2,Nissan,URVAN,2019,Manual,10,15500000.0
3,Hyundai,TUCSON,2019,Automático,5,12500000.0
4,Honda,CRV EXL,2018,Automático,5,16950000.0
...,...,...,...,...,...,...
57,Mercedes Benz,ML300,2015,Automático,5,1240272.0
58,Audi,E-TRON 55 QUATRO,2021,Automático,7,2824800.0
59,Toyota,RAV4,2005,Automático,5,1641552.0
60,Maserati,LEVANTE S MY17,2017,Manual,5,2300000.0
